## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
import joblib
import pandas as pd
from functions import to_df

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

mr = project.get_model_registry()

## <span style="color:#ff5f27">⚙️ Feature View Retrieval </span>

In [ ]:
feature_view = fs.get_feature_view(
    name='price_fv',
    version=1,
)

## <span style="color:#ff5f27">🚀 Fetch the model </span>

In [ ]:
retrieved_model = mr.get_model(
    name="xgboost_price_model",
    version=1,
)
saved_model_dir = retrieved_model.download()

In [ ]:
model = joblib.load(saved_model_dir + "/xgboost_price_model.pkl")
model

## <span style="color:#ff5f27">👨🏻‍⚖️ Real-time Predictions </span>

In [ ]:
# Initialise feature view to retrieve feature vector
feature_view.init_serving(1)

# Retrieve a feature vector
feature_vector = feature_view.get_feature_vector(
    entry = {
        "id": 1,
        "date": '2023-09-26',
    }
)
feature_vector

In [ ]:
feature_vector_df = to_df(feature_vector)
feature_vector_df

In [ ]:
# Predict transformed feature vector using retrieved model
prediction_feature_vector = model.predict(feature_vector_df)
prediction_feature_vector

In [ ]:
# Retrieve feature vectors
feature_vectors = feature_view.get_feature_vectors(
    entry = [
        {"id": 1, "date": '2023-09-26'},
        {"id": 2, "date": '2023-09-26'}
    ]
)
feature_vectors

In [ ]:
feature_vectors_df = to_df(feature_vectors)
feature_vectors_df

In [ ]:
# Predict feature vectors df using retrieved model
prediction_feature_vectors = model.predict(feature_vectors_df)
prediction_feature_vectors

## <span style="color:#ff5f27">👨🏻‍⚖️ Batch Prediction </span>

In [ ]:
# Initialise feature view to retrieve batch data
feature_view.init_batch_scoring(training_dataset_version=1)

# Retrieve batch data
batch_data = feature_view.get_batch_data(
    start_time='2023-09-01',
    end_time='2023-09-26',
)
batch_data.head(3)

In [ ]:
# Predict batch data using retrieved model
batch_data.drop('date', axis=1, inplace=True)
predictions_batch = model.predict(batch_data)
predictions_batch[:10]

---